In [1]:
import json
import torch
import torch.nn as nn

# Load the training data from the JSON file
with open('D:\\Downloads\\archive (1)\\python100k_train.json', 'r') as f:
    data = json.load(f)

# Define a dictionary to map tokens to integer IDs
token_to_id = {}

# Define a function to tokenize a code snippet and convert each token to an integer ID
def tokenize(code):
    # Split the code into tokens (e.g. words and punctuation)
    tokens = code.split()

    # Convert each token to an integer ID using the token_to_id dictionary
    ids = [token_to_id.get(token, len(token_to_id)) for token in tokens]

    return ids

# Convert each code snippet to a list of integer IDs
codes = [tokenize(item['value']) for item in data]

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, (h_n, c_n) = self.lstm(x)
        out = self.fc(h_n.squeeze(0))
        return out

# Set the hyperparameters
input_size = 1
hidden_size = 128
output_size = len(token_to_id)
num_epochs = 1
batch_size = 32
learning_rate = 0.001

# Split the data into batches
inputs = []
targets = []
for code in codes:
    for i in range(len(code)-1):
        inputs.append(code[i])
        targets.append(code[i+1])
inputs = torch.LongTensor(inputs).view(-1, batch_size, input_size)
targets = torch.LongTensor(targets).view(-1, batch_size)

# Instantiate the model and the optimizer
model = LSTMModel(input_size, hidden_size, output_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i in range(0, len(inputs)-batch_size, batch_size):
        x = inputs[i:i+batch_size]
        y = targets[i:i+batch_size]
        y_pred = model(x)
        loss = nn.functional.cross_entropy(y_pred, y.flatten())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i // batch_size) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i // batch_size, len(inputs)//batch_size, loss.item()))

# Save the trained model
torch.save(model.state_dict(), "trained_lstm_model5.pt")



JSONDecodeError: Extra data: line 2 column 1 (char 4965)